<b> Connecting to Big Query and Calendar </b>

In [ ]:
# using service account only to connect to BigQuery and Calendar.
from googleapiclient.discovery import build
from datetime import timedelta
from googleapiclient.http import BatchHttpRequest
from google.oauth2 import service_account
from google.cloud import bigquery
from pandas_gbq import to_gbq
from googleapiclient.errors import HttpError
import time
import datetime as datetime
import pandas as pd  
import numpy as np

from sentence_transformers import SentenceTransformer, util
from rapidfuzz.fuzz import ratio  

In [2]:
SERVICE_CREDENTIALS = 'service_credentials_2.json'

# Define both scopes for Calendar and BigQuery
SCOPES = [
    "https://www.googleapis.com/auth/calendar",  # Google Calendar
    "https://www.googleapis.com/auth/bigquery"   # BigQuery
]

# Create credentials with both scopes
credentials = service_account.Credentials.from_service_account_file(SERVICE_CREDENTIALS, scopes=SCOPES)

# Create BigQuery client and Calendar service using the same credentials.
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
calendar_service = build('calendar', 'v3', credentials=credentials)

<b> Checking Big Query Connection </b>

In [3]:
project_id = credentials.project_id    # or just write:  'circular-maxim-436817-e3'
dataset_id = 'test_dataset'
table_id_new = 'new_events'
full_table_id = f"{project_id}.{dataset_id}.{table_id_new}"
table_id_old = 'old_events'  # a table to store old_events (existing events).

def test_bigquery_connection(client):
    query = f"""
    SELECT COUNT(*)
    FROM `{project_id}.{dataset_id}.__TABLES_SUMMARY__`
    """
    try:
        query_job = client.query(query)
        result = next(query_job.result())  # Fetch result to ensure query runs
        print("BigQuery connection successful. Number of tables:", result[0])
    except Exception as e:
        print("Error connecting to BigQuery:", e)

# Test the BigQuery connection
test_bigquery_connection(client)

BigQuery connection successful. Number of tables: 3


In [4]:
def convert_to_yyyy_mm_dd(date_str):
    # Assume current year if not provided
    current_year = str(pd.Timestamp.now().year)
    # Split the date string into day and month
    parts = date_str.split('.')
    if len(parts) == 3 and parts[0] and parts[1]:
        # Return the date in yyyy-mm-dd format
        return f"{current_year}-{parts[1].zfill(2)}-{parts[0].zfill(2)}"
    else:
        # Handle unexpected format
        return None

In [36]:
convert_to_yyyy_mm_dd('01.01.')

['01', '01', '']
3


'2024-01-01'

In [31]:
df = pd.read_csv('Scraped_Events_Rausgegangen_HH_KI_HL_FL.csv')

df.head(2)

,Unnamed: 0,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,0,Christian Löffler - Hamburg Docks,29.11.,29.11.,19:00,NaN,Docks,Hamburg,https://rausgegangen.de/events/christian-loffl...,Konzerte & Musik,music
1,1,Christian Löffler - präsentiert von Rausgegangen,29.11.,29.11.,19:00,NaN,Docks,Hamburg,https://rausgegangen.de/events/christian-loffl...,Konzerte & Musik,music


### Preprocessing

In [32]:
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('Category', axis=1)
# Apply the function to the Start_date column
df['Start_date'] = df['Start_date'].apply(convert_to_yyyy_mm_dd)
df['End_date'] = df['End_date'].apply(convert_to_yyyy_mm_dd)

print('shape of a dataframe: ', df.shape)

shape of a dataframe:  (299, 9)


In [34]:
create_table_query = f"""
CREATE OR REPLACE TABLE `{full_table_id}` (
  `Subject` STRING,
  `Start_date` DATE,  -- Defines Start_date as DATE type.
  `End_date` DATE,    -- Defines Endt_date as DATE type.
  `Start_time` STRING,
  `End_time` STRING,
  `Location` STRING,
  `Description` STRING,
  `City` STRING,
  `Music_label` STRING
)"""
query_job = client.query(create_table_query).result()
print("Table created successfully.")

Table created successfully.


In [33]:
# Truncate the table
query = f"TRUNCATE TABLE `{full_table_id}`"

# Execute the query
query_job = client.query(query)  # Make an API request.
query_job.result()  # Wait for the query to finish.

print(f"Table {full_table_id} has been cleared.")

Table lucky-reactor-443308-r4.test_dataset.new_events has been cleared.


In [35]:
def uploading_table_to_big_query(df):    # Uploading data to BigQuery.
    try:
        to_gbq(df, destination_table=full_table_id, project_id=project_id, if_exists='append', credentials=credentials, progress_bar=True)
        print("Data uploaded to BigQuery successfully!")   #  append doesnt change table structure.
    except Exception as e:
        print(f"An error occurred: {e}")

uploading_table_to_big_query(df)

df

100%|██████████| 1/1 [00:00<?, ?it/s]

Data uploaded to BigQuery successfully!


,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Music_label
0,Christian Löffler - Hamburg Docks,2024-11-29,2024-11-29,19:00,NaN,Docks,Hamburg,https://rausgegangen.de/events/christian-loffl...,music
1,Christian Löffler - präsentiert von Rausgegangen,2024-11-29,2024-11-29,19:00,NaN,Docks,Hamburg,https://rausgegangen.de/events/christian-loffl...,music
2,Joe Astray + Anna Wydra | Living Room Sessions,2024-11-29,2024-11-29,19:00,NaN,Elbdeich Studio,Hamburg,https://rausgegangen.de/events/joe-astray-livi...,music
3,WELCOME TO SENTIMENT VALLEY Tour | Hamburg,2024-11-29,2024-11-29,19:00,NaN,Indra Musikclub,Hamburg,https://rausgegangen.de/events/welcome-to-sent...,music
4,Fabian Müller | ProArte Hamburg,2024-11-29,2024-11-29,19:30,NaN,Laeiszhalle Hamburg,Hamburg,https://rausgegangen.de/events/fabian-muller-p...,music
...,...,...,...,...,...,...,...,...,...
294,LE FLY - LIVE 2026,2024-12-12,2024-12-12,20:00,NaN,Kulturwerkstatt Kühlhaus,Flensburg,https://rausgegangen.de/events/le-fly-live-202...,music
295,LE FLY - LIVE,2024-12-12,2024-12-12,20:00,NaN,Kulturwerkstatt Kühlhaus,Flensburg,https://rausgegangen.de/events/le-fly-live-202...,music
296,LE FLY LIVE 2024,2024-12-12,2024-12-12,20:00,NaN,Kulturwerkstatt Kühlhaus,Flensburg,https://rausgegangen.de/events/le-fly-live-202...,music
297,LIVE 2024 LE FLY,2024-12-12,2024-12-12,20:00,NaN,Kulturwerkstatt Kühlhaus,Flensburg,https://rausgegangen.de/events/le-fly-live-202...,music


In [69]:
loading_query = f"SELECT * FROM `{full_table_id}`"

df_load = client.query(loading_query).to_dataframe()  # Execute the query and convert to DataFrame.

In [70]:
df_load.shape

(299, 9)

In [73]:
df_load.to_csv('Scraped_Events_Rausgegangen_HH_KI_HL_FL_new.csv')

In [72]:
import torch  # Import PyTorch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz.fuzz import ratio
import pandas as pd

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
model = AutoModel.from_pretrained("deepset/gbert-base")

# Ensure the 'Subject' column is lowercase
df['Subject'] = df['Subject'].str.lower()

# Function to generate embeddings for the 'Subject' column
def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embeddings
    return outputs.last_hidden_state[:, 0, :].numpy()

# Generate embeddings for the 'Subject' column
df['event_embedding'] = list(generate_embeddings(df['Subject'].tolist()))

# Initialize a list to store dropped rows
dropped_rows = []

# Prepare for duplicate removal
to_drop = set()

# Group by 'Start_date', 'City'
for _, group in df.groupby(['Start_date', 'City']):
    group = group.sort_values(by='Subject').reset_index()  # Sort within the group

    # Compare rows within the group
    for i in range(len(group)):
        if group.loc[i, 'index'] in to_drop:
            continue
        for j in range(i + 1, len(group)):
            if group.loc[j, 'index'] in to_drop:
                continue

            # Compute cosine similarity
            cos_similarity = cosine_similarity(
                [group.loc[i, 'event_embedding']], [group.loc[j, 'event_embedding']]
            )[0][0]

            # Compute Levenshtein similarity
            lev_similarity = ratio(group.loc[i, 'Subject'], group.loc[j, 'Subject']) / 100.0

            # Check if either similarity exceeds threshold
            if cos_similarity > 0.8 or lev_similarity > 0.85:
                to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal
                dropped_rows.append(group.loc[j].to_dict())  # Add dropped row as a dictionary

# Remove duplicates
df = df.drop(index=to_drop).reset_index(drop=True)

# Create a DataFrame for dropped rows
dropped_df = pd.DataFrame(dropped_rows)

# Drop unnecessary columns
df = df.drop(columns=['event_embedding'])

# Print the shape and dropped rows
print(f"Original DataFrame Shape: {len(df) + len(dropped_df)}")
print(f"Remaining DataFrame Rows: {len(df)}")
print(f"No. of Dropped Rows: {len(dropped_df)}")

# Display dropped rows for inspection
if not dropped_df.empty:
    print("Dropped Events:")
else:
    print("No duplicates were found.")

Original DataFrame Shape: 110
Remaining DataFrame Rows: 110
No. of Dropped Rows: 0
No duplicates were found.


In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
model = AutoModel.from_pretrained("deepset/gbert-base")

df['Subject'] = df['Subject'].str.lower()

# Encode the 'Subject' column to generate embeddings
df_load['event_embedding'] = model.encode(df['Subject'].tolist()).tolist()

# Initialize a list to store dropped rows
dropped_rows = []

# Prepare for duplicate removal
to_drop = set()

# Group by 'Start_date', 'Start_time', and 'City'
for _, group in df_load.groupby(['Start_date', 'City']):
    group = group.sort_values(by='Subject').reset_index()  # Sort within the group
    
    # Compare rows within the group
    for i in range(len(group)):
        if group.loc[i, 'index'] in to_drop:
            continue
        for j in range(i + 1, len(group)):
            if group.loc[j, 'index'] in to_drop:
                continue
            
            # Compute cosine similarity
            cos_similarity = util.cos_sim(group.at[i, 'event_embedding'], group.at[j, 'event_embedding']).item()

            # Compute Levenshtein similarity
            lev_similarity = ratio(group.at[i, 'Subject'], group.at[j, 'Subject']) / 100.0

            # Check if either similarity exceeds threshold
            if cos_similarity > 0.8 or lev_similarity > 0.85:  
                to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal
                dropped_rows.append(group.loc[j].to_dict())  # Add dropped row as a dictionary


# Remove duplicates
df_load = df_load.drop(index=to_drop).reset_index(drop=True)

# Create a DataFrame for dropped rows
dropped_df = pd.DataFrame(dropped_rows)

# Drop unnecessary columns
df_load = df_load.drop(columns=['event_embedding'])

# Print the shape and dropped rows
print(f"Original DataFrame Shape: {len(df_load) + len(dropped_df)}")
print(f"Remaining DataFrame Rows: {len(df_load)}")
print(f"No. of Dropped Rows: {len(dropped_df)}")

# Display dropped rows for inspection
print("Dropped Events:")
print(dropped_df[['Subject', 'Start_date', 'Start_time', 'City']].head())

AttributeError: 'BertModel' object has no attribute 'encode'

In [23]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # 384 dimensional dense vector space.

# Encode the 'Subject' column to generate embeddings
df_load['event_embedding'] = model.encode(df['Subject'].tolist()).tolist()

# Sort the dataframe by 'Subject', 'Start_date', and 'Start_time'
df_load = df_load.sort_values(by=['Subject', 'Start_date', 'Start_time', 'City']).reset_index(drop=True)

# Prepare for duplicate removal
to_drop = set()

# Group by 'Start_date' and 'Start_time'
for _, group in df_load.groupby(['Start_date', 'Start_time', 'City']):
    group = group.sort_values(by='Subject').reset_index()  # Sort within the group
    
    # Compare rows within the group
    for i in range(len(group)):
        if group.loc[i, 'index'] in to_drop:
            continue
        for j in range(i + 1, len(group)):
            if group.loc[j, 'index'] in to_drop:
                continue
            
            # Compute cosine similarity
            cos_similarity  = util.cos_sim(group.at[i, 'event_embedding'], group.at[j, 'event_embedding']).item()

            # Compute Levenshtein similarity
            lev_similarity = ratio(group.at[i, 'Subject'], group.at[j, 'Subject']) / 100.0

            # Check if either similarity exceeds threshold
            if cos_similarity  > 0.8 and lev_similarity > 0.9:  
                to_drop.add(group.loc[j, 'index'])  # Mark duplicate for removal

# Remove duplicates
df_load = df_load.drop(index=to_drop).reset_index(drop=True)

# Drop unnecessary columns
df_load = df_load.drop(columns=['event_embedding'])
print(df_load.shape)
df_load.head(1)

(290, 9)


,Subject,Start_date,End_date,Start_time,End_time,Location,Description,City,Music_label
0,"""Zwischen Puls & Takt"" Lesung mit Musik by HEI...",2024-12-09,2024-12-09,19:00,None,Finder‘s Haus | Raum Oslo,https://rausgegangen.de/events/zwischen-puls-t...,Lübeck,music


In [16]:
print(df_load.shape)

(289, 9)


In [13]:
df_load

,Subject,Start_date,End_date,Start_time,End_time,Location,Description,City,Music_label
0,"""Zwischen Puls & Takt"" Lesung mit Musik by HEI...",2024-12-09,2024-12-09,19:00,None,Finder‘s Haus | Raum Oslo,https://rausgegangen.de/events/zwischen-puls-t...,Lübeck,music
1,#SELFIE,2024-12-28,2024-12-28,23:59,None,Luna Club,"https://rausgegangen.de/events/selfie-12/ , Pr...",Kiel,music
2,$oho Bani,2024-12-05,2024-12-05,20:00,None,Sporthalle Hamburg,"https://rausgegangen.de/events/ohi-bani-0/ , P...",Hamburg,music
3,1. Kinderkonzert,2024-12-03,2024-12-03,09:00,None,Theater Lübeck,https://rausgegangen.de/events/1nbspkinderkonz...,Lübeck,music
4,120 Minuten Party - SUPERFAST!,2024-12-12,2024-12-12,21:00,None,Luna Club,https://rausgegangen.de/events/120-minuten-par...,Kiel,music
...,...,...,...,...,...,...,...,...,...
284,so soon,2024-12-05,2024-12-05,19:00,None,hansa48,"https://rausgegangen.de/events/so-soon-5/ , Pr...",Kiel,music
285,wavvyboi - sonst bleibt immer alles gleich tou...,2024-12-03,2024-12-03,19:00,None,Mojo Club,https://rausgegangen.de/events/sonst-bleibt-im...,Hamburg,music
286,zeck /// daydream therapy tour,2024-12-08,2024-12-08,19:00,None,Mojo Club,https://rausgegangen.de/events/zeck-daydream-t...,Hamburg,music
287,«stadtklang» m. BEN P WILLIAMS @ RUBY LOTTI HO...,2024-12-05,2024-12-05,20:00,None,Ruby Lotti Hotel & Bar,https://rausgegangen.de/events/stadtklang-m-be...,Hamburg,music


In [26]:
from datetime import datetime
df_load['Load_date'] = datetime.today().strftime('%d-%m-%Y')
df_load.head(1)

,Subject,Start_date,End_date,Start_time,End_time,Location,Description,City,Music_label,Load_date
0,"""Zwischen Puls & Takt"" Lesung mit Musik by HEI...",2024-12-09,2024-12-09,19:00,None,Finder‘s Haus | Raum Oslo,https://rausgegangen.de/events/zwischen-puls-t...,Lübeck,music,02-12-2024


In [29]:
df_load['Description'] = "Ladedatum: " + df_load['Load_date'] + ", " + df_load['Music_label'] + ", " + df_load['Description'] + ", " + df_load['City']
df_load.head(1)

,Subject,Start_date,End_date,Start_time,End_time,Location,Description,City,Music_label,Load_date
0,"""Zwischen Puls & Takt"" Lesung mit Musik by HEI...",2024-12-09,2024-12-09,19:00,None,Finder‘s Haus | Raum Oslo,"Ladedatum: 02-12-2024, music, https://rausgega...",Lübeck,music,02-12-2024


In [30]:
df_load['Description'][0]

'Ladedatum: 02-12-2024, music, https://rausgegangen.de/events/zwischen-puls-takt-lesung-mit-musik-by-heimlicht-0/ , Preis: 20,00 €, Lübeck'

<b> Uploading data back to Big Query for event comparison </b>

In [62]:
create_table_query = f"""
create or replace table `{full_table_id}` (
  `Subject` string,
  `Start_date` date,  -- Defines Start_date as DATE type.
  `End_date` date,    -- Defines Endt_date as DATE type.
  `Start_time` string,
  `End_time` string,
  `Location` string,
  `Description` string,
  `City` string,
  `Category` string,
  `Load_date` string
)"""
query_job = client.query(create_table_query).result()
print("Table created successfully.")

uploading_table_to_big_query(df)  # this function uploads data to Big Query.

Table created successfully.


100%|██████████| 1/1 [00:00<?, ?it/s]

Data uploaded to BigQuery successfully!


In [29]:
# Check if old events exist in our database:
check_query = f"""
SELECT COUNT(*) as table_count
FROM `{project_id}.{dataset_id}.__TABLES_SUMMARY__`
WHERE table_id = '{table_id_old}'
"""

check_job = client.query(check_query).result()
table_exists = next(check_job)['table_count'] > 0

# If table doesn't exist, insert the new events in it from the new_events table.
if not table_exists:
    print(f"The table '{table_id_old}' doesn't exist. Let's create one.")
    insert_query = f"""
    CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{table_id_old}` AS
    SELECT * FROM `{project_id}.{dataset_id}.{table_id_new}`
    """
    client.query(insert_query)
    print('All done')

### <b> Important step </b>

In [63]:
# Send new events in the "old_events" table.
replace_query = f"""
CREATE OR REPLACE TABLE `{project_id}.{dataset_id}.{table_id_old}` AS
SELECT * FROM `{project_id}.{dataset_id}.{table_id_new}`
"""

try:
    query_job = client.query(replace_query)  # Start the query job
    query_job.result()  # Wait for the job to complete
    print(f"The table '{table_id_old}' has been replaced with data from '{table_id_new}'.")
except Exception as e:
    print(f"An error occurred: {e}")

The table 'old_events' has been replaced with data from 'new_events'.


In [64]:
# The big event comparison thing.
query = f"""
with old_events as (
    select *,
    rank() over (partition by Subject order by Start_date, Start_time) as Event_rank
    from (
        select *,
        from {project_id}.{dataset_id}.{table_id_old}
        where End_date >= current_date()
        )
    
    ),
today_events as (
    select *,
    rank() over (partition by Subject order by Start_date, Start_time) as Event_rank
    from (
        select *,
        from {project_id}.{dataset_id}.{table_id_old}
        where End_date >= current_date()
        )
    )

    select 
    coalesce(t.Subject, p.Subject) AS Subject,       -- Capture events from either today or yesterday
    coalesce(t.Location, p.Location) AS Location,    -- Use today's location if available, otherwise yesterday's
    coalesce(t.`Start_date`, p.`Start_date`) AS Start_date,
    coalesce(t.`End_date`, p.`End_date`) AS End_date,
    coalesce(t.`Start_time`, p.`Start_time`) AS Start_time,
    coalesce(t.`End_time`, p.`End_time`) AS End_time,
    coalesce(t.Description, p.Description) AS Description,
    coalesce(t.City, p.City) AS City,  

    -- Description based on changes
    CASE 
        WHEN t.Subject IS NULL AND p.Subject IS NOT NULL THEN CONCAT(p.Description, ' | event deleted')  
        WHEN t.Location != p.Location         THEN CONCAT(t.Description, ' | previous location: ', p.Location)
        WHEN t.`Start_date` != p.`Start_date` AND t.`Start_time` != p.`Start_time` THEN 
        CONCAT(t.Description, ' | previous date : ', FORMAT_DATE('%d.%m.%Y', p.`Start_date`), ' at: ', p.`Start_time`)
        WHEN t.`Start_date` != p.`Start_date` THEN 
        CONCAT(t.Description, ' | previous date: ', FORMAT_DATE('%d.%m.%Y', p.`Start_date`))
        WHEN t.`Start_time` != p.`Start_time` THEN CONCAT(t.Description, ' | previous time: ', p.`Start_time`)
        ELSE t.Description
    END AS Updated_Description,

    -- Status to track changes: changed, unchanged or deleted events.
    CASE 
        WHEN t.Subject IS NULL AND p.Subject IS NOT NULL THEN 'deleted'  -- Event present yesterday but missing today
        WHEN t.Location != p.Location OR t.`Start_date` != p.`Start_date` OR t.`Start_time` != p.`Start_time` THEN 'changed'
        ELSE 'unchanged'
    END AS status


    from today_events t
    full outer join old_events p                        -- p stands for previous (old events).
    on t.Subject = p.Subject and t.Event_rank = p.Event_rank and t.City = p.City
    where coalesce(t.`End_date`, p.`End_date`) >= current_date()
"""
# execute the query.
query_job = client.query(query).result()

In [ ]:
success_count = 0
failure_count = 0

for i, row in enumerate(query_job):

    start_date_str = row['Start_date']
    end_date_str = row['End_date']
    start_time_str = row['Start_time']
    end_time_str = row['End_time'] 

                # Determine the color based on the 'status' column.
    if row['status'] == 'changed':
        color_id = '5'   # Yellow color for changed events.
    elif row['status'] == 'deleted':
        color_id = '11'  # Red color for deleted events.
    else:
        color_id = '10'  # Green color for unchanged events.

    # Check if start_time is NaN
    if pd.isna(start_time_str):
        # Event is all day, use only date
        start_date_obj = pd.to_datetime(start_date_str).date()  # Convert to date
        end_date_obj = pd.to_datetime(end_date_str).date()      # Convert to date

        # Adjust for events that span midnight
        if end_time_obj <= start_time_obj:
            end_time_obj += timedelta(days=1)  # Move end time to the next day. [29.11.2024]

        event = {
            'summary': row['Subject'],
            'location': row['Location'],
            'start': {
                'date': start_date_obj.isoformat(),  # Use date for all-day event
            },
            'end': {
                'date': end_date_obj.isoformat(),  # End date for all-day event
            },
            'description': row['Updated_Description'],  # Include the prepared description
            'colorId': color_id  # Set the color based on the status
        }
    else:
        # Combine date and time for start
        start_datetime_str = f"{start_date_str}T{start_time_str}"
        start_time_obj = pd.to_datetime(start_datetime_str)

        # Combine date and time for end, or add a default duration if end time is None
        if pd.isna(end_time_str):  # If 'End_time' is None
            end_datetime_str = f"{end_date_str}T{start_time_str}"
            end_time_obj = pd.to_datetime(end_datetime_str) + timedelta(minutes=27)
        else:
            end_datetime_str = f"{end_date_str}T{end_time_str}"
            end_time_obj = pd.to_datetime(end_datetime_str)

        # Adjust for events that span midnight
        if end_time_obj <= start_time_obj:
            end_time_obj += timedelta(days=1)  # Move end time to the next day. [29.11.2024]

        # if start_time_obj >= end_time_obj:  # Check for valid time range
        #     print(f"Error: Invalid time range for row {i}. Start: {start_time_obj}, End: {end_time_obj}")
        #     continue  # Skip this event
        
        # Prepare the event details for Google Calendar:
        event = {
            'summary': row['Subject'],
            'location': row['Location'],
            'start': {
                'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            },
            'end': {
                'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            },
            'description': row['Updated_Description'],  # Include the prepared description
            'colorId': color_id  # Set the color based on the status
        }
    try:
        created_event = calendar_service.events().insert(calendarId='foks243dabl4@gmail.com', body=event).execute()
        print(f"Event created: {created_event.get('htmlLink')}")
        success_count += 1
    except Exception as e:
        print(f"Error creating event: {e}")
        failure_count += 1

print(f"success count: {success_count}")
print(f"failure count: {failure_count}")

Event created: https://www.google.com/calendar/event?eid=OWFrNjM4bmQ3aDA2MG5ibGgxN2Z2bDFwdXMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=dmc3OWcybWwyaGE2MjV2NWs3ZjBjYnB1MDggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=ZnZuMTdha2JpYXVjYnU1bHBuZ2YwOXJuMDAgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aDhhYzZkMTVxYWpkdDg4NmJyMHV1cWQwYnMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=N3Vobjh2MHIzazRpdmQ0cmI5b2hkaDhmZ2cgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=bTBwa2o4Yjd1YTdwOXAwcjJqODVvNG1ydjggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=NWJ1OXE2b2k1bmRtYmc2a3MybTdtcDZ0OWMgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=aDNkdDgycjd2NDZubHVjNG90MDgyNWs4YTggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=NDFlN3Fobm9vMXA4NXZldDhsNzg3dWdqcjAgZm9

In [66]:
print(f"success count: {success_count}")

success count: 111


The deleting events should also take place.